In [1]:
import pandas as pd
from report_manager import project, dataset, report, analyses, plots
import networkx as nx
from networkx.readwrite import json_graph
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
from scipy.stats import zscore
init_notebook_mode(connected=True)
%matplotlib inline
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

import warnings
warnings.filterwarnings('ignore')
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

/Users/plh450/CKG_env_updated/lib/python3.6/site-packages/rpy2/robjects/pandas2ri.py:197: UserWarning:

R object inheriting from "POSIXct" but without attribute "tzone".

Using TensorFlow backend.
/Users/plh450/CKG_env_updated/lib/python3.6/site-packages/rpy2/robjects/pandas2ri.py:197: UserWarning:

R object inheriting from "POSIXct" but without attribute "tzone".



In [83]:
my_project = project.Project(identifier='P0000001', datasets={}, report={})
my_project.load_project_data()
my_project.load_project_report()

In [84]:
proteomics_dataset = my_project.get_dataset('proteomics')

In [85]:
my_project.list_datasets()

dict_keys(['multiomics', 'clinical', 'proteomics'])

In [86]:
my_project.get_dataset('proteomics').list_dataframes()

['correlation_correlation',
 'go annotation',
 'number of modified proteins',
 'number of peptides',
 'number of proteins',
 'original',
 'pathway annotation',
 'processed',
 'protein biomarkers',
 'regulated',
 'regulation table']

In [87]:
proteomics_dataset = my_project.get_dataset('proteomics')
processed_df = proteomics_dataset.get_dataframe('processed')

In [88]:
processed_df.head()

,A2M~P01023,A30~A2MYE2,ABI3BP~Q7Z7G0,ACE~P12821,ACTB~P60709,ACTN1~P12814,ADA2~Q9NZK5,ADAMTS13~Q76LX8,ADAMTSL4~Q6UY14,ADH4~P08319,...,VIM~P08670,VK3~A2N2F4,VNN1~O95497,VTN~P04004,VWF~P04275,YWHAZ~P63104,group,sample,scFv~Q65ZC9,subject
0,38.005564,28.173504,21.540133,22.165420,27.090330,25.039968,23.442151,24.010605,25.085820,23.389032,...,24.178889,25.835908,22.480055,32.815815,28.922779,19.198487,Cirrhosis,AS68,27.788928,S68
1,37.309118,27.981907,27.342062,23.847270,27.461155,25.896268,23.754503,24.135818,19.183590,22.148706,...,23.709777,25.004889,23.852908,32.722121,29.881279,22.141285,Cirrhosis,AS69,26.869972,S69
2,37.384952,28.857627,20.123246,22.863630,27.929764,24.295225,23.359443,24.121788,24.923476,23.017163,...,23.599064,26.271650,24.232132,32.755752,29.444625,18.869370,Cirrhosis,AS70,28.069328,S70
3,38.417225,28.978380,25.501910,22.992774,27.152479,25.231288,23.701340,24.568309,24.878802,26.388112,...,24.179076,25.929200,24.269047,32.714014,29.397176,22.216971,Cirrhosis,AS71,28.170209,S71
4,37.471303,28.748744,21.906161,22.203381,27.537048,22.392992,22.406264,24.961173,21.995507,24.339540,...,23.865224,26.701340,20.755495,32.722691,28.540895,20.880564,Cirrhosis,AS72,28.612280,S72


In [89]:
processed_df = processed_df.drop(['sample', 'subject'], axis=1).set_index('group').apply(zscore).reset_index()

In [90]:
proteomics_report = my_project.get_dataset('proteomics').report

In [100]:
# proteomics_report.list_plots()

In [92]:
#proteomics_report.visualize_plot(environment='notebook', plot_type='13~correlation_correlation~network')

In [93]:
correlation_net_dict = proteomics_report.get_plot('15~correlation_correlation~network')[0]

In [94]:
correlation_net = json_graph.node_link_graph(correlation_net_dict['net_json'])

In [95]:
correlation_df = pd.DataFrame.from_dict(correlation_net.nodes(data=True))
correlation_df = correlation_df[0].to_frame().join(correlation_df[1].apply(pd.Series))

In [96]:
correlation_df.columns = ['identifier', 'degree', 'radius', 'color', 'cluster']

In [97]:
min_val = processed_df._get_numeric_data().min().min().round()
max_val = processed_df._get_numeric_data().max().max().round()
processed_df = processed_df[list(correlation_df.identifier) + ['group']]

In [98]:
from IPython.core.display import display, HTML

In [99]:
@interact
def plot_parallel_plot(cluster=correlation_df.cluster.unique()):
    cluster_colors = dict(zip(correlation_df.cluster, correlation_df.color))
    clusters = correlation_df.groupby('cluster')
    identifiers = clusters.get_group(cluster)['identifier'].tolist()
    title= "Parallel plot cluster: {}".format(cluster)
    df = processed_df.set_index('group')[identifiers].reset_index()
    print(df.head())
    figure = plots.basicFigures.get_parallel_plot(df, identifier=cluster, args={'color':cluster_colors[cluster], 'group':'group', 
                                                                          'title':title,
                                                                          'zscore':False})
    display(HTML("<p>{}</p>".format(",".join(identifiers))))
    iplot(figure.figure)

interactive(children=(Dropdown(description='cluster', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 1…